# Ghost 7B Alpha

<p><img src="https://cdn-uploads.huggingface.co/production/uploads/600ae38cc92b79f54efd4556/QPrQZMQX_jzyYngmreP0_.jpeg" width="40%" align="center" /></p>

The large generation of language models focuses on optimizing excellent reasoning, multi-task knowledge, and tools support.

### Introduction

**Ghost 7B Alpha** is a large language model fine-tuned from Mistral 7B, with a size of 7 billion parameters. The model was developed with the goal of optimizing reasoning ability, multi-task knowledge and supporting tool usage. The model works well with the main trained and optimized languages being English and Vietnamese.

Overall, the model is suitable when making a pretrained version so you can continue to develop the desired tasks, develop virtual assistants, perform features on tasks such as coding, translation, answering questions, creating documents, etc. It is truly an efficient, fast and extremely cheap open model.

See details about the model [here](https://ghost-x.vercel.app/docs/models/ghost-7b-alpha/).

# Setup

First, to start experiencing the **Ghost 7B Alpha** model we need to install the necessary libraries, here will be transformers and related things.

## Unsloth

Finetune Llama 3, Mistral & Gemma LLMs 2-5x faster with 80% less memory. [[source]](https://github.com/unslothai/unsloth)

### Key Features

- All kernels written in OpenAI's Triton language. Manual backprop engine.
- 0% loss in accuracy - no approximation methods - all exact.
- No change of hardware. Supports NVIDIA GPUs since 2018+. Minimum CUDA Capability 7.0 (V100, T4, Titan V, RTX 20, 30, 40x, A100, H100, L40 etc) Check your GPU! GTX 1070, 1080 works, but is slow.
- Works on Linux and Windows via WSL.
- Supports 4bit and 16bit QLoRA / LoRA finetuning via bitsandbytes.
- Open source trains 5x faster - see Unsloth Pro for up to 30x faster training!

## Installation

Make the installation easy by running the following command.

In [ ]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

## Downloads & More

Then, create and load the model from **HuggingFace** to use. In the code below, it will load and minify the text generation using functions created to condense the code for later use.

In [ ]:
import os, torch, pprint
from transformers import pipeline
from unsloth import FastLanguageModel

base_model = "ghost-x/ghost-7b-alpha"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model,
    max_seq_length = 8192,
    dtype = None,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
)

def generate_sample_response(
    messages,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
):
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = pipe.tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=False,
    )
    for k,v in inputs.items():
        inputs[k] = v.cuda()
    outputs = pipe.model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )
    results = pipe.tokenizer.batch_decode(outputs)[0]
    return results


def generate_default_response(
    messages,
    max_new_tokens=512,
):
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = pipe.tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=False,
    )
    for k,v in inputs.items():
        inputs[k] = v.cuda()
    outputs = pipe.model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
    )
    results = pipe.tokenizer.batch_decode(outputs)[0]
    return results

In [ ]:
# import torch
# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)

# Playground

Now let's start to experience the model.

In the code below, we will generate text with the model without setting a system prompt for the model. These will represent generic use cases.

In [ ]:
question = """\
In your opinion, what trends will shape the world in the next 5 years? Explain the reasons for your choice.
"""

messages = [
    {"role": "system", "content": ""},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

Additionally, to try out the model with other role plays, start with the fun ones below.

### Try science and physics skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **physics expert**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Describe the difference between potential energy and kinetic energy to a 12 year old child. Give examples of each type.
"""

messages = [
    # {"role": "system", "content": "You are Albert Einstein. You can easily explain complex things in the field of science and physics."},
    # {"role": "system", "content": "You are a scientist. You always answer questions accurately with explanations."},
    {"role": "system", "content": "You are a physics expert. You always use your knowledge to answer questions accurately and understandably."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
A physical phenomenon is characterized by the transfer of energy without the movement of matter. It can occur in a vacuum and does not require a medium to travel through. This phenomenon is responsible for making us see distant objects, such as stars and planets.

What physical phenomenon is it?
"""

messages = [
    {"role": "system", "content": "You are a scientist. You always answer questions accurately with explanations."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Explain alternating current. Give an easy-to-understand example.
"""

messages = [
    {"role": "system", "content": "You are Nikola Tesla."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try coding skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **software engineer**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Write code to solve the following problem: Given a list of integers, find the position of the largest and smallest number in the list, in Python.
"""

messages = [
    # {"role": "system", "content": "You are a software engineer."},
    # {"role": "system", "content": "You are a software engineer with 5 years of experience working at Google."},
    # {"role": "system", "content": "You are a computer science engineer. You have extensive knowledge of artificial intelligence and machine learning."},
    {"role": "system", "content": "You are a software engineer. You have extensive knowledge of computer science. You always answer questions accurately and professionally."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Explain how to use the TensorFlow library in Python to build and train an artificial neural network for the text classification problem.
"""

messages = [
    {"role": "system", "content": "You are a AI engineer. You have extensive knowledge of artificial intelligence and machine learning."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try reasoning skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **science expert**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
The Case of the Puzzling Poison
A wealthy socialite, known for her extravagant parties and lavish lifestyle, is found dead at her opulent mansion. The police are baffled by the circumstances of her death, as there are no signs of forced entry or struggle, and the victim's only known enemies are her many disgruntled ex-lovers.
As the lead detective on the case, you are called in to investigate. Upon arriving at the mansion, you are greeted by a scene of opulence and disarray. The victim lies sprawled on the floor of her grand dining room, a half-empty glass of champagne clutched in her hand. A quick examination reveals that she died from a lethal dose of poison.
The room is a mess, with chairs overturned and silverware scattered across the table. However, there are no signs of forced entry, and the only window is securely locked from the inside. The victim's dinner guests, a group of her closest friends and business associates, are all present and seem genuinely shocked by her death.
As you interview the guests, you learn that the victim had been arguing with her business partner, a shrewd and ruthless businessman, just days before her death. The argument was reportedly heated, and the two had not spoken since. However, the business partner denies any involvement in the victim's death and claims to have an alibi for the time of her murder.
You also learn that the victim had recently hired a new butler, a young man with a mysterious past. The butler was the only person who had access to the victim's food and drinks, and he had served her the champagne just moments before she died. However, the butler swears that he is innocent and that he has no reason to harm the victim.
With no clear suspects and no obvious signs of foul play, you are left scratching your head. The case seems unsolvable, until you notice a small, almost imperceptible detail that the other investigators have overlooked. This seemingly insignificant detail holds the key to unlocking the mystery and identifying the killer.

What is the small detail that the detective notices?
"""

messages = [
    # {"role": "system", "content": "You are the famous detective Conan. You help users solve difficult cases with your excellent reasoning and logic abilities."},
    # {"role": "system", "content": "You are a professional detective. You reason to find the cause of problems logically."},
    {"role": "system", "content": "You are a science expert. You are very good at reasoning to answer challenging questions."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
A scientist is conducting an experiment to determine the effect of different temperatures on the growth of bacteria. The scientist prepares five identical Petri dishes and adds the same amount of bacteria to each dish. The scientist then places each dish in a different incubator set to a different temperature. After 24 hours, the scientist measures the growth of the bacteria in each dish. The results are shown in the following table:

Temperature (°C)	Bacterial growth (mm)
20	5
30	10
40	15
50	10
60	5

Based on the data, what can the scientist conclude about the effect of temperature on the growth of bacteria?
"""

messages = [
    {"role": "system", "content": "You are a science expert. You are very good at reasoning to answer challenging questions."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Mysterious case at the old villa:

The victim was Mrs. Lan Anh, a wealthy woman, who was found dead in her study in an ancient mansion. Police arrived at the scene and found some clues:

On Ms. Lan Anh's desk there is an unsent letter, handwritten and saying: "I have found out your secret. Regret it!".
Next to Ms. Lan Anh's body was a precious, broken antique vase.
The fingerprint of a servant named Long was found on the broken vase.
Long testified that he was cleaning the study room the morning of the incident and accidentally bumped into the vase, causing it to break. However, he affirmed that he never saw Ms. Lan Anh and did not know what happened.
There were no signs of forced entry or struggle at the scene.
Question:

Who is the culprit who murdered Ms. Lan Anh? Based on the clues collected, make logical inferences and come up with a reasonable explanation for this mysterious case.
"""

messages = [
    {"role": "system", "content": "You are Sherlock Holmes. You always answer logically with your brilliant reasoning."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try financial skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **financial experts**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
In your opinion, what factors affect the value of Vietnamese currency compared to other currencies?
"""

messages = [
    # {"role": "system", "content": "You are the wolf of wall street. You always advise to help users make a lot of profit with low risk."},
    {"role": "system", "content": "You are a financial expert. You always give advice carefully, after carefully examining the facts."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Below is information about the types of bank cards VIB is providing.

| Card                      | VIB Super Card                    | VIB Cash Back                  | VIB Premier Boundless              |
| ------------------------- | --------------------------------- | ------------------------------ | ---------------------------------- |
| Transaction Limit         | Up to 600 million VND             | Up to 600 million VND          | Up to 2 billion VND                |
| Key Features              |                                   |                                |                                    |
|                           | Flexible refund up to 15%...      |                                |                                    |
|                           | Minimum payment rate and...       |                                |                                    |
|                           | Free selection of beautiful...    |                                |                                    |
|                           | Up to 24 million bonus points...  |                                |                                    |
|                           | Accumulate Vietnam Airlines...    |                                |                                    |
|                           | Gift 4,000 Bong Sen Vang...       |                                |                                    |
|                           | Free upgrade to Titan, Gold,...   |                                |                                    |
| Annual Fee                | 999,000 VND                       | 899,000 VND                    | 1,299,000 VND                      |
| Interest Rate             | 1st 3 statement cycles:...        | 2.83% per month                | 2.83% per month                    |
|                           | Subsequent statement cycles:...   | Interest-free up to 55 days    | Interest-free up to 55 days        |
| Cashback, Rewards Program | Refund up to 15% on spending...   | Unlimited cash back            | Refund up to 10% for registered... |
|                           |                                   | Refund 0.1% for total valid... |                                    |
|                           | Accumulate reward miles for...    |                                |                                    |
|                           | 25,000 VND spending at Vietnam... |                                |                                    |
|                           |                                   |                                |                                    |
| Redeeming Rewards         | You can actively convert...       | You can actively convert...    | Reward miles are automatically...  |

Question: I am a project manager, work often traveling, income is about 40 million VND/month. Please advise me on the appropriate card type.
"""

messages = [
    {"role": "system", "content": "You are a banking consultant. You are always honest and give the most beneficial and appropriate advice to users."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try literature and art skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as an **expert in literature and art**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
What character in Shakespearean literature is known for saying, "To be or not to be, that is the question" ?
"""

messages = [
    # {"role": "system", "content": "You are a famous artist. You have extensive knowledge of works of art."},
    # {"role": "system", "content": "You are a famous literary researcher. You can answer your knowledge of literature accurately."},
    {"role": "system", "content": "You are an expert in literature and art."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.2,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Which of your works are you most impressed with? Explain why ?
If you could change a part of the content, how would you change it?
"""

messages = [
    {"role": "system", "content": "You are William Shakespeare."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try medical and health skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as an **medical experts**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Patient: Male, 35 years old, came to the doctor because he had a high fever of 39°C continuously for 3 days, accompanied by a dry cough, headache, body aches and fatigue. The patient has no history of underlying medical conditions and is not allergic to drugs.

Clinical examination:
Temperature: 39°C
Blood pressure: 120/80 mmHg
Pulse: 90 times/minute
Breathing rate: 20 times/minute
Lung examination: rattling sounds in both lungs
Heart: regular heartbeat, no abnormal heart sounds
Abdomen: soft, painless
Skin: no rash
Question:

Based on the above information, what disease do you think the patient may have? Please explain the reasons for your choice and suggest the necessary tests for an accurate diagnosis.
"""

messages = [
    # {"role": "system", "content": "You are a health expert. You like to help and always give accurate health and nutrition information."},
    {"role": "system", "content": "You are a medical professional. You always give scientific and useful advice."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.2,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
If you could invent a medicine, what medicine would you choose to invent?
"""

messages = [
    {"role": "system", "content": "You are Alexandre Yersin."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.2,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)